# 资源分配问题建模

考虑三个工作岗位：测试员、Java开发人员和架构师。

考虑三个资源：Carlos、Joe和Monika。

## 数据

每个资源执行每项工作的能力由以下匹配分数表说明：

![资源分配问题数据图](util/rap_data.png)


**假设**：每个工作只能分配一个资源，每个资源只能分配一个工作。

## 问题陈述

确定一种分配方案，确保每项工作都能得到完成，每个资源最多分配一项工作，目的是使分配的总匹配分数最大化。

## 决策变量

决策变量 $x_{r,\; j} = 1$ 表示资源r被分配给工作j，否则为0，其中r=1,2,3且j=1,2,3。

## 约束条件

### 工作约束

对于每个工作j=1,2,3，必须从r=1,2,3中分配恰好一个资源。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

约束（Java开发人员=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

### 资源约束

对于每个资源r=1,2,3，最多只能分配一项工作。

约束（Carlos=1）：$x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

约束（Joe=2）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

约束（Monika=3）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

## 目标函数

目标函数是在满足工作和资源约束的前提下，最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




首先，根据需要安装一些包

In [ ]:
%pip install gurobipy
%pip install names
%pip install numpy

In [1]:
# 导入gurobi库
from gurobipy import *

## 数据
列表R包含三个资源的名称：Carlos、Joe和Monika。

列表J包含工作岗位的名称：测试员、Java开发人员和架构师。

**数学表示法**

$r \in R$意味着索引为r的资源在集合（列表）R中。

$j \in J$意味着索引为j的工作在集合（列表）J中。

In [2]:
# 资源和工作集合
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

以下“multidict”函数描述了每种资源和工作组合可能关联的匹配分数。

**数学表示法**

设$ms_{r,\;j}$为资源$r \in R$对于工作$j \in J$的匹配分数。

设$C_{r,\;j}$为将资源$r \in R$分配给工作$j \in J$的成本。

设$B$为可用预算。

In [3]:
# 匹配分数数据
combinations, ms, C = multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# 可用预算
#B = 6
B=5

以下函数生成一个空模型对象“m”，并将字符串“RAP”模型名称作为参数。

In [4]:
# 声明并初始化模型
m = Model('RAP')

Set parameter LicenseID to value 2601452


## 决策变量

决策变量$x_{r,\; j} = 1$表示资源r被分配给工作j，否则为0，其中r=1,2,3且j=1,2,3。

“addVars()”方法定义了模型对象“m”的决策变量。

**数学表示法**

设$x_{r,\; j} = 1$，如果资源$r \in R$被分配给工作$j \in J$，否则为零。

设$g_{j} = 1$，如果工作$j \in J$无法填补，否则为零。这个变量是一个缺口变量，表示工作无法被填补。


In [5]:
# 为RAP模型创建决策变量
#x = m.addVars(combinations, name="assign")
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# 为RAP模型创建缺口变量
g = m.addVars(J, name="gap")

## 工作约束

对于每个工作j=1,2,3，必须从r=1,2,3中分配恰好一个资源。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

约束（Java开发人员=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

“addConstrs()”方法定义了模型对象“m”的约束条件。

**数学表示法**

对于每个工作$j \in J$，必须分配恰好一个资源：

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1 
$$



In [6]:
# 创建工作约束
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), 'job')

## 资源约束

对于每个资源r=1,2,3，最多只能分配一项工作。

约束（Carlos=1）：$x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

约束（Joe=2）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

约束（Monika=3）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

“addConstrs()”方法定义了模型对象“m”的约束条件。

**数学表示法**

对于每个资源$r \in R$，最多只能分配一项工作：

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1 
$$

In [7]:
# 创建资源约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

## 预算约束

将资源分配给工作的总成本应小于或等于可用预算。

$$
\sum_{r \; \in \; R} \sum_{j \; \in \; J} C_{r, j}x_{r, j} \leq B
$$

In [8]:
budget = m.addConstr((x.prod(C) <= B), 'budget')

## 目标函数

目标函数是最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

“setObjective()”方法定义了模型对象“m”的目标函数。

**数学表示法**

注意
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

因此，目标函数可以表示为

$$
Max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j} -BigM \sum_{j \in J} g_{j}
$$


In [9]:
# 未能填补工作岗位的惩罚
BIGM =101

# 目标是最大化分配的总匹配分数
m.setObjective(x.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)

In [ ]:
# 保存模型以供检查
m.write('RAP3.lp')

In [10]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160

In [11]:
# 显示决策变量的最优值
for v in m.getVars():
	if (abs(v.x) > 1e-6):
		print(v.varName, v.x)

# 显示最优总匹配分数
print('最优目标函数值', m.objVal)

# 从分配变量计算总匹配分数
total_matching_score = 0
for [r, j] in combinations:
    if (abs(x[r, j].x) > 1e-6):
        total_matching_score = total_matching_score + ms[r, j]*x[r, j].x

print('总匹配分数: ', total_matching_score)


assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
gap[Architect] 1.0
最优目标函数值 52.0
总匹配分数:  153.0


## 生成随机场景

In [13]:
import names
import random
import numpy as np
from gurobipy import *
from itertools import product

def generate_scenario(num_resources=200, num_jobs=200, roles=None,
                      score_mu=50, score_sigma=15, seed=10101):
    random.seed(seed)
    np.random.seed(seed)
    if roles is None:
        roles = {"Architect", "BackEndEngineer", "FrontEndEngineer",
                     "Tester", "DataScientist", "DataEngineer"}
    # 资源成本的概率密度函数遵循本福特定律，取值范围为{1,2,...,9}
    benford = [np.log10((i+1)/i) for i in range(1,10)]
    # 抽样资源名称
    resources = {names.get_full_name() for i in range(num_resources)}
    # 抽样工作需求，假设所有角色被选中的可能性相等
    req = np.random.multinomial(num_jobs, [1/len(roles)]*len(roles), size=1)[0]
    jobs = set()
    # 为每个工作岗位分配ID
    for i, role in enumerate(roles):
        jobs = jobs.union(set(map(''.join, zip([role]*req[i], [str(x).zfill(int(np.log10(num_jobs))+1) for x in range(1,req[i]+1)]))))
    scores = {}
    costs = {}
    # 为每个潜在分配抽样匹配分数和成本
    for pair in product(resources, jobs):
        scores[pair] = int(np.clip(np.random.normal(score_mu, score_sigma), 0, 100))
        costs[pair] = random.choices(list(range(1,10)), weights=benford, k=1)[0]
    return resources, jobs, scores, costs 

In [14]:
res, job, ms, cst = generate_scenario(seed=11111)
budget = 200

## 获取贪心解

In [19]:
def greedy_solve(resources, jobs, scores, costs, budget):
    assign = set()
    total_score = 0
    remaining_budget = budget
    while remaining_budget > 0 and len(scores.keys()) > 0:
        selection = max(scores, key=scores.get)
        assign.add(selection)
        total_score += scores[selection]
        remaining_budget -= costs[selection]
        # 删除与新选择相关的资源/工作的潜在分配
        res_filter = list(filter(lambda x: x[0] == selection[0], scores))
        job_filter = list(filter(lambda x: x[1] == selection[1], scores))
        blacklist = res_filter + job_filter
        scores = {key: val for key,val in scores.items()
                  if key not in blacklist
                  and costs[key] <= remaining_budget}
    print("分配数量: {0}".format(len(assign)))
    print("总匹配分数: {0}".format(total_score))
    print("已用预算: {0}".format(budget - remaining_budget))
    
    kpi = {}
    kpi["n_assign"] = len(assign)
    kpi["total_ms"] = total_score
    kpi["budget_used"] = budget - remaining_budget
    return assign, kpi
        

In [20]:
greedy_sol, kpi = greedy_solve(res, job, ms, cst, budget)

# 贪心启发式KPI
Greedy_assign = kpi["n_assign"]
Greedy_ms = kpi["total_ms"]

#print('贪心分配数量: ', Greedy_assign)
#print('贪心总匹配分数: ',Greedy_ms)

分配数量: 58
总匹配分数: 5589
已用预算: 200


# 获取最优解

In [22]:
m = Model("RAP")
assign = m.addVars(ms.keys(), vtype=GRB.BINARY, name="assign")
g = m.addVars(job, name="gap")
m.addConstrs((assign.sum("*", j) + g[j]  == 1 for j in job), name="demand")
m.addConstrs((assign.sum(r, "*") <= 1 for r in res), name="supply")
m.addConstr(assign.prod(cst) <= budget, name="Budget")
m.setObjective(assign.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 401 rows, 40200 columns and 120200 nonzeros
Model fingerprint: 0xcca97cf6
Variable types: 200 continuous, 40000 integer (40000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -20200.00000
Presolve time: 0.13s
Presolved: 401 rows, 40200 columns, 120200 nonzeros
Variable types: 0 continuous, 40200 integer (40200 binary)

Root relaxation: objective 1.627500e+04, 592 iterations, 0.05 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0           

In [23]:
def print_solution(model):
    i = 1
    total_ms = 0
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}) {1}: {2}".format(i, var.varName, var.x))
            i += 1
            if "assign" in var.varName:
                total_ms += var.Obj
    print('总匹配分数: {0}'.format(total_ms))
    print('最优目标函数值: {0}'.format(model.objVal))
    return None

# 显示决策变量的最优值
print_solution(m)

1) assign[Sharon Dean,DataScientist020]: 1.0
2) assign[Jennifer Wohner,FrontEndEngineer004]: 1.0
3) assign[Bianca Harris,Tester039]: 1.0
4) assign[Maria Beals,Architect021]: 1.0
5) assign[Jay Terry,Tester031]: 1.0
6) assign[Robin Ransome,DataScientist019]: 1.0
7) assign[Anita Smith,FrontEndEngineer017]: 1.0
8) assign[Richard Jennings,Architect022]: 1.0
9) assign[Jennie James,DataEngineer028]: 1.0
10) assign[Eric Gonzalez,Tester024]: 1.0
11) assign[Tressie Vargas,DataEngineer007]: 1.0
12) assign[Jeffrey Dixon,BackEndEngineer020]: 1.0
13) assign[James Carter,Tester025]: 1.0
14) assign[Kristin Wermers,BackEndEngineer002]: 1.0
15) assign[Charles Thomas,Tester040]: 1.0
16) assign[John Nester,Tester041]: 1.0
17) assign[John Thomas,DataScientist011]: 1.0
18) assign[Odell Deanda,DataEngineer015]: 1.0
19) assign[Earl Rosen,DataScientist010]: 1.0
20) assign[Lance Segura,Tester022]: 1.0
21) assign[Heather Khan,BackEndEngineer006]: 1.0
22) assign[Thomas Kaufman,FrontEndEngineer003]: 1.0
23) assign

In [27]:
# 比较贪心启发式和Gurobi优化器的KPI
Gurobi_assign = 0
Gurobi_ms = 0
for [r,j] in ms.keys():
    if (abs(assign[r, j].x) > 1e-6):
        Gurobi_assign = Gurobi_assign + assign[r, j].x
        Gurobi_ms = Gurobi_ms + ms[r, j]*assign[r, j].x
        
print('Gurobi总分配数量: ', Gurobi_assign)
print('Gurobi总匹配分数: ', Gurobi_ms)

#贪心分配数量与Gurobi分配数量的比率
assign_ratio = 100*Greedy_assign/Gurobi_assign
ms_ratio = 100*Greedy_ms/Gurobi_ms

print('分配比率: ',"%.2f" % assign_ratio, '%')
print('匹配分数比率: ',"%.2f" % ms_ratio, '%')

Gurobi总分配数量:  200.0
Gurobi总匹配分数:  16275.0
分配比率:  29.00 %
匹配分数比率:  34.34 %
